In [2]:
import sys
import os

# Add the project root directory to sys.path
project_root = os.path.abspath(".") 
if project_root not in sys.path:
    sys.path.append(project_root)
print(sys.path[-1])


from reward_func.evo_devo import coord_reward_func, oscillator_reward_func, somitogenesis_reward_func


/home/dannyhuang/gfn_test/discrete-gflownet


In [3]:
import time

test_state = (50, -53, -57, 8, 9, -6, -117, 81, 8)

start_time = time.perf_counter_ns()
test_reward = coord_reward_func(test_state)
end_time = time.perf_counter_ns()
print(f"Time taken to run coord_reward_func: {(end_time - start_time)/1e9:.9f} seconds")

print(f"Test reward for state {test_state}: {test_reward}")
    



# test_weights = [-5, 2, 3, 1, 2, -1, 2, 1, -3] 
# test_weights = [0, -10, 0, 4, 6, -10, -10, 0, 0]  # w12=-5, w22=..., w23=-10, w31=-10 
# test_weights = [154, -200, 82, -33, 90, -90, -82, -13, 30] 
# reward = oscillator_reward_func(test_weights, plot=True)
# print(f"Reward for oscillator: {reward}")

Time taken to run coord_reward_func: 0.000024570 seconds
Test reward for state (50, -53, -57, 8, 9, -6, -117, 81, 8): 4


In [ ]:
# Import plotting function from graph module
from graph.graph import draw_network_motif
import matplotlib.pyplot as plt

# test_weights = (126, -125, -56, 107, 105, -126, 100, -11, 175, 1, 1, 1, 1, 1, 1, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25) 
# test_weights = (-6, -25, 0, 5, -25, 25, 0, 0, 200, 25, -200, 0, 0, 0, -1, 0, 0, 0, 0, 0, 15, 0, 5, 0, 0, 25, 0, 5, 0, 0, 0, -25, -25, 1, 0, 0, 1, -25, 0, 10, 10, 0, -5, -25, 0, 5, 1, -1, 0)

test_weights = [60, 32, -38, -85, 70, -63, 22, -27, -7]
test_weights = [60, 32, -38, -85, 70, -63, 22, -27, -7, -1, -11, 12, 13, 14, 0, 1]
test_weights = [1, 2, 8, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 16]

test_weights = [200, 0, 0, 25, 0, 0, -200, 25, 0]
test_weights = [-100, 0, 0, -100, 0, 0, -100, 0, 0]

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 12))

# Plot network motif
draw_network_motif(test_weights, ax=ax1)
ax1.set_title("Network Motif")

# Plot somite pattern and get reward
start_time = time.perf_counter_ns()
reward = somitogenesis_reward_func(test_weights, plot=True, ax=ax2)
print(f"Reward for somitogenesis: {reward}")
end_time = time.perf_counter_ns()
print(f"Time taken to run somitogenesis_reward_func: {(end_time - start_time)/1e9:.9f} seconds")

plt.tight_layout()


In [ ]:
# Import plotting function from graph module
from graph.graph import plot_network_motifs_and_somites

# Define test weights to visualize
test_weights_list = [
    # (65, -110, 52, -40, 32, -8, -65, -32, 71),
    [-5, 200, -6, -51, -26, 5, 5, 1, -125, 25, 30, 100, 0, 60, -5, -25, -1, 5, 0, -75, 30, 0, -1, -200, -25],
    [60, 32, -38, -85, 70, -63, 22, -27, -7],
    [60, 32, -38, -85, 70, -63, 22, -27, -7, 0, 0, 0, 0, 0, 0, 0]
]

# Plot network motifs and their corresponding somite patterns
save_path = plot_network_motifs_and_somites(test_weights_list)
print(f"Plot saved to: {save_path}")



In [15]:
# import time

# # Function that simulates an expensive computation
# def expensive_computation(x):
#     time.sleep(0.1)  # Simulate work
#     return x * 2

# # Test without cache
# print("Without cache:")
# start_time = time.time()
# for i in range(10):
#     # Compute same values multiple times
#     result = expensive_computation(i % 3)  # Only compute 0,1,2 repeatedly
#     print(f"Computing {i % 3} -> {result}")
# no_cache_time = time.time() - start_time
# print(f"Time without cache: {no_cache_time:.2f}s\n")

# # Test with cache
# print("With cache:")
# cache = LRUCache(max_size=3)
# start_time = time.time()
# for i in range(10):
#     key = i % 3
#     if key in cache:
#         result = cache[key]
#         print(f"Cache hit for {key} -> {result}")
#     else:
#         result = expensive_computation(key)
#         cache[key] = result
#         print(f"Cache miss for {key} -> {result}")
# cache_time = time.time() - start_time
# print(f"Time with cache: {cache_time:.2f}s")

# print(f"\nCache speedup: {no_cache_time/cache_time:.1f}x faster")


In [ ]:
import numpy as np
import time
from numba import njit, prange, set_num_threads, threading_layer
import os

import multiprocessing

print("CPU cores available:", multiprocessing.cpu_count())
set_num_threads(4)  # Manually limit to 4 cores




# Slow Python loop
def compute_squares_python(arr):
    return [x**2 for x in arr]

# Fast Numba compiled loop (Single-threaded)
@njit  # Just-in-time compilation
def compute_squares_numba(arr):
    result = np.empty_like(arr)
    for i in range(len(arr)):  # Regular loop (single-threaded)
        result[i] = arr[i] ** 2
    return result

# Faster Numba compiled loop (Parallelized)
@njit(parallel=True)
def compute_squares_parallel(arr):
    result = np.empty_like(arr)
    for i in prange(len(arr)):
        result[i] = arr[i] ** 2
    return result

# Test the speed
arr = np.arange(10**9)

# # Python loop
# start = time.time()
# compute_squares_python(arr)  
# print("Python loop time:", time.time() - start)

# Numba single-threaded
start = time.time()
compute_squares_numba(arr)  
print("Numba single-threaded time:", time.time() - start)

# Numba parallel
start = time.time()
compute_squares_parallel(arr)  
print("Numba parallel time:", time.time() - start)



In [11]:
import argparse
from disc_gflownet.envs.grid_env import GridEnv
import numpy as np

# Test cases for Gray code encoding/decoding
args = argparse.Namespace(
    min_reward=0.001,
    custom_reward_fn=coord_reward_func,
    n_workers = 1, 
    cache_max_size = 10000, 
    consistent_signs = True, 
    n_steps=2,
    n_dims=2,  # Testing with 1 dimension
    actions_per_dim=[-1, 1],  # Mixed positive and negative actions
    grid_bound=10,
    enable_time=False
)
env = GridEnv(args)
print('encoding_dim: ', env.encoding_dim)

# Test case 1: Basic encoding/decoding without time
test_state = np.array([-9, -10])  # Single dimension state
encoded = env.state_to_encoding(test_state)
decoded = env.encoding_to_state(encoded)
print("Test 1 - Basic encoding/decoding:")
print(f"Original state: {test_state}")
print(f"Encoded: {encoded}")
print(f"Decoded state: {decoded}")
print(f"Match: {np.array_equal(test_state, decoded)}\n")

# # Test case 2: Test one step
# env.reset()
# env.print_actions()
# obs, reward, done = env.step(2) 
# print("Test 2 - One step:")
# print(f"Observation after step: {obs}")
# print(f"Current state: {env._state}\n")
# obs, reward, done = env.step(6) 
# print("Test 2 - One step:")
# print(f"Observation after step: {obs}")
# print(f"Current state: {env._state}\n")



encoding_dim:  42
Test 1 - Basic encoding/decoding:
Original state: [ -9 -10]
Encoded: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]
Decoded state: [ -9 -10]
Match: True

